In [1]:
!pip install diffusers transformers torch pillow librosa speechrecognition matplotlib numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 54.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found

In [2]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [3]:
%%writefile /kaggle/working/mood_mapping.json
{
  "happy": {
    "colors": ["yellow", "orange"],
    "lighting": "bright, sunny",
    "style": "vibrant, cartoon"
  },
  "sad": {
    "colors": ["blue", "gray"],
    "lighting": "dim, overcast",
    "style": "muted, photorealistic"
  },
  "calm": {
    "colors": ["light blue", "pastel green"],
    "lighting": "soft, diffused",
    "style": "serene, watercolor"
  },
  "angry": {
    "colors": ["red", "black"],
    "lighting": "harsh, dramatic",
    "style": "bold, cinematic"
  },
  "excited": {
    "colors": ["bright red", "yellow"],
    "lighting": "dynamic, vivid",
    "style": "energetic, illustrative"
  }
}

Writing /kaggle/working/mood_mapping.json


In [4]:
!ls /kaggle/working

mood_mapping.json


In [ ]:
import json
import random
import os
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
from IPython.display import display
from kaggle_secrets import UserSecretsClient
import ipywidgets as widgets
from IPython.display import clear_output

# Access Hugging Face token
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

# Load mood mapping
def load_mood_mapping(file_path="/kaggle/working/mood_mapping.json"):
    try:
        with open(file_path, "r") as f:
            return json.load(f)
    except FileNotFoundError:
        print("Error: mood_mapping.json not found.")
        return {}

# Generate prompt from mood
def generate_prompt(mood, mood_mapping):
    if mood not in mood_mapping:
        return None, f"Sorry, I don't have '{mood}' in my mood list. Try: {', '.join(mood_mapping.keys())}"
    mood_attrs = mood_mapping[mood]
    color = random.choice(mood_attrs["colors"])
    lighting = mood_attrs["lighting"]
    style = mood_attrs["style"]
    subject = "scene"  # Default subject
    prompt = f"a {mood} {subject}, {color} tones, {lighting} lighting, {style} style"
    return prompt, None

# Generate image using Stable Diffusion
def generate_image(prompt, output_path="/kaggle/working/mood_image.png"):
    try:
        # Initialize Stable Diffusion
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16,
            use_auth_token=hf_token
        )
        # Move to GPU
        pipe = pipe.to("cuda")
        # Generate image
        image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]
        # Save image
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        image.save(output_path)
        return output_path, None
    except Exception as e:
        return None, f"Error generating image: {str(e)}"

# Create interactive widget interface
def create_chatbot_interface():
    mood_mapping = load_mood_mapping()
    if not mood_mapping:
        print("Cannot start chatbot: mood_mapping.json not found.")
        return

    # Create header with instructions
    header = widgets.HTML(
        value='<h2 style="text-align: center; color: #2c3e50; font-family: Arial;">MoodSense Art Bot</h2>'
              '<p style="text-align: center; color: #34495e; font-family: Arial;">Enter a mood (e.g., happy, sad, calm) to generate a unique image!</p>'
    )

    # Create text input for mood
    mood_input = widgets.Text(
        value="happy",
        placeholder="Enter a mood (e.g., happy, sad)",
        description="Mood:",
        style={'description_width': 'initial'},
        layout={'width': '400px', 'margin': '10px 0'}
    )

    # Create generate button
    generate_button = widgets.Button(
        description="Generate Image",
        button_style="success",
        tooltip="Click to generate image",
        icon="image",
        layout={'width': '200px', 'margin': '10px 0', 'padding': '10px'}
    )

    # Create output area
    output = widgets.Output(layout={'border': '1px solid #dfe6e9', 'padding': '10px', 'margin': '10px 0'})

    # Define button click handler
    def on_generate_button_clicked(b):
        with output:
            clear_output()  # Clear previous output
            mood = mood_input.value.lower().strip()
            print(f"Processing mood: {mood}")
            prompt, error = generate_prompt(mood, mood_mapping)
            if error:
                print(error)
                return

            print(f"Generated prompt: {prompt}")
            output_path, error = generate_image(prompt)
            if error:
                print(error)
            else:
                print(f"Image generated! Saved to: {output_path}")
                try:
                    img = Image.open(output_path)
                    display(img)
                except Exception as e:
                    print(f"Could not display image: {str(e)}")

    # Link button to handler
    generate_button.on_click(on_generate_button_clicked)

    # Organize UI with VBox
    ui = widgets.VBox(
        [header, mood_input, generate_button, output],
        layout={'align_items': 'center', 'padding': '20px', 'background_color': '#f8f9fa'}
    )

    # Display interface
    display(ui)

# Run the chatbot interface
create_chatbot_interface()